In [1]:
from hw3align.sequences import *
from hw3align.getmatrix import blosum50
from hw3align.getmatrix import blosum62
from hw3align.getmatrix import pam100
from hw3align.getmatrix import pam250
from hw3align.getmatrix import matio
from hw3align.getmatrix import matio
#from hw3align.getmatrix import optblosum50
from hw3align.optimalmatrix import *
from hw3align.optimalgaps import *
from hw3align.smithwaterman import sw
from hw3align.getseq import *
from hw3align.roc import *

from hw3align.optimalmatrix import *
import sys
import matplotlib.pyplot as plt
import sklearn.metrics as skm

In [ ]:
#optimizeMatrix_geneticAlg(blosum50, mutationChance, mutationAmount,
	#selectivePressure, N, totalStepsToStop, stepsWithNoImprovement,
	#librarySize, gap_start, gap_extend, true_pos_align, true_neg_align):

In [2]:
true_pos_align, true_neg_align = getAlignments(scorematrix=blosum50, gap_start=-8, gap_extend= -3)

In [3]:
true_pos_align2, true_neg_align2 = getAlignments(scorematrix=matio, gap_start=-8, gap_extend= -3)

In [4]:
pop, scores, library, objectiveMeans = optimizeMatrix_geneticAlg(blosum50, 1, 1, 2, 10, 5, 5, 3, -8, -3, true_pos_align, true_neg_align)



In [5]:
print(scores)

[2.12, 2.04, 2.2199999999999998, 2.2199999999999998, 2.04, 2.1, 2.1, 2.4000000000000004, 2.36, 2.38]


In [6]:
print(objectiveMeans)

[2.0740000000000003, 1.952, 2.186, 2.19, 2.2540000000000004, 2.198]


In [7]:
pop2, scores2, library2, objectiveMeans2 = optimizeMatrix_geneticAlg(blosum50, 1, 1, 2, 10, 5, 5, 3, -8, -3, true_pos_align, true_neg_align)




In [8]:
print(scores2)

[2.1799999999999997, 1.9600000000000002, 2.1799999999999997, 2.1799999999999997, 1.9000000000000001, 2.0, 1.98, 2.3600000000000003, 2.2, 2.1]


In [9]:
print(objectiveMeans2)

[2.0919999999999996, 2.096, 2.0340000000000003, 2.124, 2.062, 2.104]


In [ ]:
print(len(objectiveMeans))